In [49]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
# import altair as alt
# alt.renderers.enable('notebook')

In [50]:
train_data = pd.read_csv("poverty/src/data/train.csv")

# Impute Missing Value

According to the following table, we know that variables rez_esc, v18q1, v2a1, SQBmeaned, and meaneduc contain missing value. <br>
<br>
**rez_esc**: Years behind in school<br>
**v18q1**: number of tablets household owns<br>
**v2a1**: Monthly rent payment<br>
**SQBmeaned**: square of the mean years of education of adults (>=18) in the household<br>
**meaneduc**: average years of education for adults (18+)<br>

In [51]:
# Number of missing in each column
missing = pd.DataFrame(train_data.isnull().sum()).rename(columns = {0: 'total'})

# Create a percentage missing
missing['percent'] = missing['total'] / len(train_data)

missing.sort_values('percent', ascending = False).head(7)

,total,percent
rez_esc,7928,0.829549
v18q1,7342,0.768233
v2a1,6860,0.717798
SQBmeaned,5,0.000523
meaneduc,5,0.000523
instlevel2,0,0.000000
instlevel1,0,0.000000


## 1. rez_esc

`rez_esc` indicates years behind in school. Since the percentage of missing value is more than 80%, it will be hard to predict the missing value and the predicted value may be inaccuate. Therefore, we choose to drop this column. 

In [52]:
null_rez = train_data['rez_esc'].isnull()

In [53]:
print('The row with missing value: ' + str(sum(null_rez) / len(null_rez)))

The row with missing value: 0.82954902166


In [54]:
train_data = train_data.drop("rez_esc", axis = 1)

## 2. v18q1

In the dataset, we know that `v18q` indicates whether or not a family owns a tablet, where 0 indicates no tablet and 1 indicates own tablet. When we check v18q and v18q1, we find that when column v18q shows 0, there is a missing value in column v18q1. Then, we suppose that families with a NaN in this category just do not own a tablet. 

In [55]:
train_data[['v18q','v18q1']].head()

,v18q,v18q1
0,0,NaN
1,1,1.0
2,0,NaN
3,1,1.0
4,1,1.0


We choose to groupby the value of v18q (which is 1 for owns a tablet and 0 for does not) and then calculate the number of null values for v18q1. This will tell us if the null values represent that the family does not own a tablet.

In [56]:
null_df = train_data.groupby('v18q')['v18q1'].apply(lambda x: x.isnull().sum()).to_frame(name=None)
null_df.columns = ['num']
null_df

,num
v18q,
0,7342
1,0


Therefore, according to the result above, we know that family with NaN in v18q1 means that the family does not own a tablet. To be more specific, we can fill in the missing value with value 0. 

In [57]:
train_data['v18q1'] = train_data['v18q1'].fillna(0)

## 3. v2a1

`v2a1` represents monthly rent payment. <br>
In order to impute the missing values of the monthly rent payment, we also check the relationship between v2a1 and tipovivi which represent the ownership or renting status of the home. <br>
For this plot, we show the ownership status of those homes with a nan for the monthyl rent payment.

In [58]:
data_with_null_v2a1 = train_data.loc[train_data['v2a1'].isnull()]

In [59]:
own_variables = [x for x in data_with_null_v2a1 if x.startswith('tipo')]

In [60]:
data_with_null_v2a1[own_variables].sum().plot.bar(color = 'royalblue');
plt.xticks([0, 1, 2, 3, 4],
           ['Owns and Paid Off', 'Owns and Paying', 'Rented', 'Precarious', 'Other'],
          rotation = 60)
plt.title('Home Ownership Status for Households Missing Rent Payments');

We can find that most of family with the missing value in v2a1 "owns and paid off" their house so they do not have to pay the rent. For "Precarious" and "Other", for any reason, the families do not need to pay for the rent as well. <br>
Therefore, we will impute the missing value with value 0 in this column. 

In [61]:
train_data['v2a1'] = train_data['v2a1'].fillna(0)

## 4. SQBmeaned & meaneduc

We know that SQBmeaned is square of the meaneduc and there are only 5 rows of missing value. We choose to drop these 5 rows of data.

In [62]:
train_data = train_data.dropna()

# Combine Dummy Data

1. We find that there are two columns `tamhog` and `hhsize`both mean the size of the household, we choose to delete column `tamhog`.

In [63]:
print('The percentage of the same value in tamhog and hhsize: '+ str(len(train_data.loc[train_data['tamhog'] == train_data['hhsize']]) / len(train_data)))

The percentage of the same value in tamhog and hhsize: 1.0


In [64]:
train_data = train_data.drop("tamhog", axis = 1)

2. Since there are many dummy data in the dataset, in order to do statistical analyses conveniently, we choose to combine the dummy data.

In [65]:
def fill_outside_wall_material(row):
    if row['paredblolad'] == 1:
        return 'Block or Brick'
    elif row['paredzocalo'] == 1:
        return 'Socket'
    elif row['paredpreb'] == 1:
        return 'Prefabricated or Cement'
    elif row['pareddes'] == 1:
        return 'Waste Matrial'
    elif row['paredmad'] == 1:
        return 'Wood'
    elif row['paredzinc'] == 1:
        return 'Zink'
    elif row['paredfibras'] == 1:
        return 'Natural Fibers'
    elif row['paredother'] == 1:
        return 'Other'
    else:
        return 'Unidentifiable'

In [66]:
def fill_floor_material(row):
    if row['pisomoscer'] == 1:
        return 'Mosaic, Ceramic, Terrazo'
    elif row['pisocemento'] == 1:
        return 'Cement'
    elif row['pisoother'] == 1:
        return 'Other'
    elif row['pisonatur'] == 1:
        return 'Natural Material'
    elif row['pisonotiene'] == 1:
        return 'No Floor'
    elif row['pisomadera'] == 1:
        return 'Wood'
    else:
        return 'Unidentifiable'

In [67]:
def fill_roof_material(row):
    if row['techozinc'] == 1:
        return 'Metal Foil or Zink'
    elif row['techoentrepiso'] == 1:
        return 'Fiber Cement or Mezzanine'
    elif row['techocane'] == 1:
        return 'Natural Fibers'
    elif row['techootro'] == 1:
        return 'Other'
    elif row['cielorazo'] != 1:
        return 'No Ceiling'
    else:
        return 'Unidentifiable'

In [68]:
def fill_water_provision(row):
    if row['abastaguadentro'] == 1:
        return 'Inside the Wall'
    elif row['abastaguafuera'] == 1:
        return 'Outside the Wall'
    elif row['abastaguano'] == 1:
        return 'No Provision'
    else:
        return 'Unidentifiable'

In [69]:
def fill_electricity_source(row):
    if row['public'] == 1:
        return 'CNFL, ICE, ESPH/JASEC'
    elif row['planpri'] == 1:
        return 'Private Plant'
    elif row['noelec'] == 1:
        return 'No Electricity'
    elif row['coopele'] == 1:
        return 'Cooperative'
    else:
        return 'Unidentifiable'

In [70]:
def fill_toilet(row):
    if row['sanitario1'] == 1:
        return 'No Toilet'
    elif row['sanitario2'] == 1:
        return 'Sewer or Cesspool'
    elif row['sanitario3'] == 1:
        return 'Septic Tank'
    elif row['sanitario5'] == 1:
        return 'Black Hole or Letrine'
    elif row['sanitario6'] == 1:
        return 'Other System'
    else:
        return 'Unidentifiable'

In [71]:
def fill_cooking_energy_source(row):
    if row['energcocinar1'] == 1:
        return 'No Kitchen'
    elif row['energcocinar2'] == 1:
        return 'Electricity'
    elif row['energcocinar3'] == 1:
        return 'Gas'
    elif row['energcocinar4'] == 1:
        return 'Wood Charocal'
    else:
        return 'Unidentifiable'

In [72]:
def fill_rubbish_disposal(row):
    if row['elimbasu1'] == 1:
        return 'Tanker Trunck'
    elif row['elimbasu2'] == 1:
        return 'Botan Hollow or Buried'
    elif row['elimbasu3'] == 1:
        return 'Burning'
    elif row['elimbasu4'] == 1:
        return 'Unoccupied Space'
    elif row['elimbasu5'] == 1:
        return 'River, Creek, or Sea'
    elif row['elimbasu6'] == 1:
        return 'Other'
    else:
        return 'Unidentifiable'

In [73]:
def fill_wall_quality(row):
    if row['epared1'] == 1:
        return 'Bad'
    elif row['epared2'] == 1:
        return 'Regular'
    elif row['epared3'] == 1:
        return 'Good'
    else:
        return 'Unidentifiable'

In [74]:
def fill_roof_quality(row):
    if row['etecho1'] == 1:
        return 'Bad'
    elif row['etecho2'] == 1:
        return 'Regular'
    elif row['etecho3'] == 1:
        return 'Good'
    else:
        return 'Unidentifiable'

In [75]:
def fill_floor_quality(row):
    if row['eviv1'] == 1:
        return 'Bad'
    elif row['eviv2'] == 1:
        return 'Regular'
    elif row['eviv3'] == 1:
        return 'Good'
    else:
        return 'Unidentifiable'

In [76]:
def fill_gender(row):
    if row['male'] == 1:
        return 'Male'
    elif row['female'] == 1:
        return 'Female'
    else:
        return 'Unidentifiable'

In [77]:
def fill_civil_status(row):
    if row['estadocivil1'] == 1:
        return 'Less Than 10 Years Old'
    elif row['estadocivil2'] == 1:
        return 'Free or Coupled Union'
    elif row['estadocivil3'] == 1:
        return 'Married'
    elif row['estadocivil4'] == 1:
        return 'Divorced'
    elif row['estadocivil5'] == 1:
        return 'Separated'
    elif row['estadocivil6'] == 1:
        return 'Widow/er'
    elif row['estadocivil7'] == 1:
        return 'Single'
    else:
        return 'Unidentifiable'

In [78]:
def fill_relationship(row):
    if row['parentesco1'] == 1:
        return 'Household Head'
    elif row['parentesco2'] == 1:
        return 'Spouse/Partner'
    elif row['parentesco3'] == 1:
        return 'Son/Daughter'
    elif row['parentesco4'] == 1:
        return 'Stepson/Daughter'
    elif row['parentesco5'] == 1:
        return 'Son/Daughter in Law'
    elif row['parentesco6'] == 1:
        return 'Grandson/daughter'
    elif row['parentesco7'] == 1:
        return 'Mother/Father'
    elif row['parentesco8'] == 1:
        return 'Father/Mother in Law'
    elif row['parentesco9'] == 1:
        return 'Brother/Sister'
    elif row['parentesco10'] == 1:
        return 'Brother/Sister in Law'
    elif row['parentesco11'] == 1:
        return 'Other Family Member'
    elif row['parentesco12'] == 1:
        return 'Other Non Family Member'
    else:
        return 'Unidentifiable'

In [79]:
def fill_education_level(row):
    if row['instlevel1'] == 1:
        return 'No Level of Education'
    elif row['instlevel2'] == 1:
        return 'Incomplete Primary'
    elif row['instlevel3'] == 1:
        return 'Complete Primary'
    elif row['instlevel4'] == 1:
        return 'Incomplete Academic Secondary Level'
    elif row['instlevel5'] == 1:
        return 'Complete Academic Secondary Level'
    elif row['instlevel6'] == 1:
        return 'Incomplete Technical Secondary Level'
    elif row['instlevel7'] == 1:
        return 'Complete Technical Secondary Level'
    elif row['instlevel8'] == 1:
        return 'Undergraduate and Higher Education'
    elif row['instlevel9'] == 1:
        return 'Postgraduate Higher Education'
    else:
        return 'Unidentifiable'

In [80]:
def fill_type(row):
    if row['tipovivi1'] == 1:
        return 'Own and Fully Paid House'
    elif row['tipovivi2'] == 1:
        return 'Own, Paying in Installments'
    elif row['tipovivi3'] == 1:
        return 'Rented'
    elif row['tipovivi4'] == 1:
        return 'Precarious'
    elif row['tipovivi5'] == 1:
        return 'Other, Assigned or Borrowed'
    else:
        return 'Unidentifiable'

In [81]:
def fill_place(row):
    if row['lugar1'] == 1:
        return 'Central'
    elif row['lugar2'] == 1:
        return 'Chorotega'
    elif row['lugar3'] == 1:
        return 'Pacifico central'
    elif row['lugar4'] == 1:
        return 'Brunca'
    elif row['lugar5'] == 1: 
        return 'Huetar Atlantica'
    elif row['lugar6'] == 1:
        return 'Huetar Norte'
    else:
        return 'Unidentifiable'

In [82]:
def fill_area(row):
    if row['area1'] == 1:
        return 'Urban'
    elif row['area2'] == 1:
        return 'Rural'
    else:
        return 'Unidentifiable'

In [83]:
def combine_dummy_feature(train_data):
    train_data['outside_wall_material'] = train_data.apply (lambda row: fill_outside_wall_material(row),axis=1)
    train_data['floor_material'] = train_data.apply (lambda row: fill_floor_material(row),axis=1)    
    train_data['roof_material'] = train_data.apply (lambda row: fill_roof_material(row),axis=1)
    train_data['water_provision'] = train_data.apply (lambda row: fill_water_provision(row),axis=1)
    train_data['electricity_source'] = train_data.apply (lambda row: fill_electricity_source(row),axis=1)
    train_data['toilet'] = train_data.apply (lambda row: fill_toilet(row),axis=1)
    train_data['cooking_energy_source'] = train_data.apply (lambda row: fill_cooking_energy_source(row),axis=1)
    train_data['rubbish_disposal'] = train_data.apply (lambda row: fill_rubbish_disposal(row),axis=1)
    train_data['wall_quality'] = train_data.apply (lambda row: fill_wall_quality(row),axis=1)
    train_data['roof_quality'] = train_data.apply (lambda row: fill_roof_quality(row),axis=1)
    train_data['floor_quality'] = train_data.apply (lambda row: fill_floor_quality(row),axis=1)  
    train_data['gender'] = train_data.apply (lambda row: fill_gender(row),axis=1)
    train_data['civil_status'] = train_data.apply (lambda row: fill_civil_status(row),axis=1)
    train_data['relationship'] = train_data.apply (lambda row: fill_relationship(row),axis=1)    
    train_data['education_level'] = train_data.apply (lambda row: fill_education_level(row),axis=1)
    train_data['type'] = train_data.apply (lambda row: fill_type(row),axis=1)
    train_data['place'] = train_data.apply (lambda row: fill_place(row),axis=1)    
    train_data['area'] = train_data.apply (lambda row: fill_area(row),axis=1)
    
    return train_data

In [84]:
def drop_dummy_col(train_data):
    wall_lst = ['paredblolad','paredzocalo','paredpreb','pareddes','paredmad','paredzinc','paredfibras','paredother']
    floor_lst = ['pisomoscer','pisocemento','pisoother','pisonatur','pisonotiene','pisomadera']
    roof_lst = ['techozinc','techoentrepiso','techocane','techootro','cielorazo']
    water_provision_lst = ['abastaguadentro','abastaguafuera','abastaguano']
    electricity_lst = ['public','planpri','noelec','coopele']
    toilet_lst = ['sanitario1','sanitario2','sanitario3','sanitario5','sanitario6']
    energy_lst = ['energcocinar1','energcocinar2','energcocinar3','energcocinar4']
    disposal_lst = ['elimbasu1','elimbasu2','elimbasu3','elimbasu4','elimbasu5','elimbasu6']
    walls_quality_lst = ['epared1','epared2','epared3']
    roof_quality_lst = ['etecho1','etecho2','etecho3']
    floor_quality_lst = ['eviv1','eviv2','eviv3']
    gender_lst = ['male','female']
    status_lst = ['estadocivil1','estadocivil2','estadocivil3','estadocivil4','estadocivil5','estadocivil6','estadocivil7']
    relationship_lst = ['parentesco1','parentesco2','parentesco3','parentesco4','parentesco5','parentesco6','parentesco7','parentesco8','parentesco9',
                'parentesco10','parentesco11','parentesco12']
    education_lst = ['instlevel1','instlevel2','instlevel3','instlevel4','instlevel5','instlevel6','instlevel7','instlevel8','instlevel9']
    type_lst = ['tipovivi1','tipovivi2','tipovivi3','tipovivi4','tipovivi5']
    region_lst = ['lugar1','lugar2','lugar3','lugar4','lugar5','lugar6']
    area_lst = ['area1','area2']
    
    train_data = train_data.drop(wall_lst, axis = 1)
    train_data = train_data.drop(floor_lst, axis = 1)
    train_data = train_data.drop(roof_lst, axis = 1)
    train_data = train_data.drop(water_provision_lst, axis = 1)
    train_data = train_data.drop(electricity_lst, axis = 1)
    train_data = train_data.drop(toilet_lst, axis = 1)
    train_data = train_data.drop(energy_lst, axis = 1)
    train_data = train_data.drop(disposal_lst, axis = 1)
    train_data = train_data.drop(walls_quality_lst, axis = 1)
    train_data = train_data.drop(roof_quality_lst, axis = 1)
    train_data = train_data.drop(floor_quality_lst, axis = 1)
    train_data = train_data.drop(gender_lst, axis = 1)
    train_data = train_data.drop(status_lst, axis = 1)
    train_data = train_data.drop(relationship_lst, axis = 1)    
    train_data = train_data.drop(education_lst, axis = 1)
    train_data = train_data.drop(type_lst, axis = 1)
    train_data = train_data.drop(region_lst, axis = 1)
    train_data = train_data.drop(area_lst, axis = 1)
    
    return train_data

In [85]:
def clean_dataframe(train_data):
    clean_data = combine_dummy_feature(train_data)
    clean_data = drop_dummy_col(clean_data)
    
    return clean_data

## Gain Cleaned Data and Save Data as .csv File

In [86]:
clean_data = clean_dataframe(train_data)

In [88]:
clean_data.to_csv("poverty/src/data/poverty_data.csv", index=False)